In [ ]:
from wisconsin import data, baseline, model, optimise
from typing import List
import plotly.express as px
import shap
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# data.download(unzip=True)

In [ ]:
baseline_df = baseline.load_baseline()

In [ ]:
df = data.load()

In [ ]:
def train(
    run_name: str,
    df: pd.DataFrame,
    model_types: List[str] = ["random_forest", "logistic_regression", "svm", "xgboost", "neural_network", "lightgbm"],
    n_splits=5
):
    results = []

    for model_type in model_types:
        print(f"{model_type}...")
        result = model.cross_validate_model(df, model_type, n_splits=n_splits)
        results.append(result)
    results_df = pd.DataFrame(results)

    acc_pr_df = results_df[['model', 'accuracy_min', 'accuracy_mean', 'accuracy_max', 'precision_min', 'precision_mean', 'precision_max']].copy()
    acc_pr_df.rename(columns={"accuracy_mean": "accuracy_avg", "precision_mean": "precision_avg"}, inplace=True)
    acc_pr_df[["accuracy_min","accuracy_avg","accuracy_max","precision_min","precision_avg","precision_max"]] *= 100
    acc_pr_df['model_type'] = run_name

    acc_pr_df.replace(
        {
            "random_forest" : "random_forest_classification",
            "xgboost": "xgboost_classification",
            "svm": "support_vector_classification",
            "neural_network": "neural_network_classification"
        }, inplace=True
    )

    return results_df, acc_pr_df


In [ ]:
def get_n_colors(n, cmap_name='nipy_spectral'):
    cmap = plt.cm.get_cmap(cmap_name, n)
    return [f'rgb({int(r*255)},{int(g*255)},{int(b*255)})'
            for r, g, b, _ in cmap(np.linspace(0, 1, n))]

def precision_plot(dfs: List[pd.DataFrame], height=None, more_colours=False):
    
    df = pd.concat(dfs)

    model_order = df['model'].unique()
    model_map = {model: i for i, model in enumerate(model_order)}
    df['y_numeric'] = df['model'].map(model_map).astype(float)

    jitter_strength = 0.2
    jitter = df['model_type'].astype('category').cat.codes * jitter_strength
    df['y_jittered'] = df['y_numeric'] + jitter

    unique_models = df['model_type'].nunique()
    
    if more_colours:
        color_list = get_n_colors(unique_models)
    else:
        color_list = px.colors.qualitative.Plotly

    fig = px.scatter(
        df,
        x='precision_avg',
        y='y_jittered',
        color='model_type',
        error_x=df['precision_max'] - df['precision_avg'],
        error_x_minus=df['precision_avg'] - df['precision_min'],
        labels={'precision_avg': 'Precision (%)'},
        title='Precision',
        template='plotly_white',
        size_max=10,
        color_discrete_sequence=color_list,
    )

    fig.update_traces(marker=dict(size=10))
    fig.update_yaxes(
        tickvals=list(model_map.values()),
        ticktext=list(model_map.keys()),
        title='Model'
    )
    if height:
        fig.update_layout(
            height=height,
        )
    fig.show()


def accuracy_plot(dfs: List[pd.DataFrame], height=None, more_colours=False):
    
    df = pd.concat(dfs)

    model_order = df['model'].unique()
    model_map = {model: i for i, model in enumerate(model_order)}
    df['y_numeric'] = df['model'].map(model_map).astype(float)

    jitter_strength = 0.2
    jitter = df['model_type'].astype('category').cat.codes * jitter_strength
    df['y_jittered'] = df['y_numeric'] + jitter

    unique_models = df['model_type'].nunique()


    if more_colours:
        color_list = get_n_colors(unique_models)
    else:
        color_list = px.colors.qualitative.Plotly

    fig = px.scatter(
        df,
        x='accuracy_avg',
        y='y_jittered',
        color='model_type',
        error_x=df['accuracy_max'] - df['accuracy_avg'],
        error_x_minus=df['accuracy_avg'] - df['accuracy_min'],
        labels={'accuracy_avg': 'Accuracy (%)'},
        title='Accuracy',
        template='plotly_white',
        size_max=10,
        color_discrete_sequence=color_list,
    )

    fig.update_traces(marker=dict(size=10))
    fig.update_yaxes(
        tickvals=list(model_map.values()),
        ticktext=list(model_map.keys()),
        title='Model'
    )
    if height:
        fig.update_layout(
            height=height,
        )

    fig.show()

In [ ]:
def plot_roc_curves(df):
    models = df["model"].tolist()
    fig = make_subplots(rows=3, cols=2, subplot_titles=models)

    for i, row in df.iterrows():
        roc_data = row["roc_data"]
        model_name = row["model"]

        row_num = i // 2 + 1
        col_num = i % 2 + 1

        for fold_data in roc_data:
            fpr = fold_data["fpr"]
            tpr = fold_data["tpr"]
            fig.add_trace(
                go.Scatter(
                    x=fpr,
                    y=tpr,
                    mode="lines",
                    name=f"{model_name} - fold {fold_data['fold']}",
                    showlegend=False,
                    line=dict(width=1)
                ),
                row=row_num,
                col=col_num,
            )

        fig.add_trace(
            go.Scatter(
                x=[0, 1],
                y=[0, 1],
                mode="lines",
                line=dict(dash="dash", color="gray"),
                showlegend=False,
            ),
            row=row_num,
            col=col_num,
        )

    fig.update_layout(
        height=900,
        width=800,
        title_text="ROC Curves per Model",
    )

    fig.update_xaxes(title_text="False Positive Rate")
    fig.update_yaxes(title_text="True Positive Rate")

    fig.show()

## Rebuild Baseline Plots

In [ ]:
precision_plot([baseline_df])
accuracy_plot([baseline_df])

## Default models

Try emulating baseline models with 5-fold cross validation and default parameters.

In [ ]:
default_df, default_acc_pr_df = train(run_name="default", df=df)
precision_plot([baseline_df, default_acc_pr_df])
accuracy_plot([baseline_df, default_acc_pr_df])


In [ ]:
plot_roc_curves(default_df)

## Feature Importance

Compute Shapley values to analyse contribution of features to predictions.

- refit model with whole data and random_forest model
- compute and plot Shapley values

In [ ]:
m = model.refit_model(df, model_name="random_forest")

X = df.drop(columns=["ID", "Diagnosis"])

explainer = shap.Explainer(m, X)
shap_values = explainer(X)

shap_values_class1 = shap_values.values[:, :, 1]

shap_values_class1_expl = shap.Explanation(
    values=shap_values_class1,
    base_values=shap_values.base_values[:, 1],
    data=X,
    feature_names=X.columns.tolist()
)

mean_abs_shap = np.abs(shap_values_class1).mean(axis=0)
feature_names = X.columns

shap_df = pd.DataFrame({
    "Feature": feature_names,
    "Mean SHAP Value": mean_abs_shap
}).sort_values(by="Mean SHAP Value", ascending=False)

top_k = 10
top_df = shap_df.iloc[:top_k].copy()

if shap_df.shape[0] > top_k:
    others_sum = shap_df.iloc[top_k:]["Mean SHAP Value"].sum()
    other_row = pd.DataFrame([{
        "Feature": "All other features",
        "Mean SHAP Value": others_sum
    }])
    top_df = pd.concat([top_df, other_row], ignore_index=True)

fig = px.bar(
    top_df,
    x="Mean SHAP Value",
    y="Feature",
    orientation="h",
    title=f"Top {top_k} Features by Mean SHAP Value (with 'All other features')",
    labels={"Mean SHAP Value": "Mean |SHAP value|"},
    height=600
)

fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

## Top 5 Features Model

Try using only the top 5 features (for the RF model above) for building models.

In [ ]:
top5_df, top5_acc_pr_df = train(run_name="top_5_features", df=df[["Diagnosis","area3", "perimeter3", "concave_points3", "radius3", "concave_points1"]])
precision_plot([baseline_df, default_acc_pr_df, top5_acc_pr_df])
accuracy_plot([baseline_df, default_acc_pr_df, top5_acc_pr_df])


In [ ]:
plot_roc_curves(top5_df)


## Top 1 Features Model

Try using only the top feature (for the RF model above) for building models.

In [ ]:
top1_df, top1_acc_pr_df = train(run_name="top_feature", df=df[["Diagnosis","area3"]])
precision_plot([baseline_df, default_acc_pr_df, top5_acc_pr_df, top1_acc_pr_df])
accuracy_plot([baseline_df, default_acc_pr_df, top5_acc_pr_df, top1_acc_pr_df])


In [ ]:
plot_roc_curves(top1_df)

## One Feature Only

Neural network with just a single feature per run

In [ ]:
one_feat_acc_pr_dfs = [default_acc_pr_df[default_acc_pr_df["model"] == "neural_network_classification"]] 

for feature in df.drop(columns=["ID", "Diagnosis"]).columns:
    print(feature)

    feat_df, feat_acc_pr_df = train(run_name=f"nn_{feature}", df=df[['Diagnosis', feature]], model_types=["neural_network"])
    one_feat_acc_pr_dfs.append(feat_acc_pr_df)

one_feat_acc_pr_df = pd.concat(one_feat_acc_pr_dfs)

one_feat_acc_pr_df = one_feat_acc_pr_df.sort_values(by="precision_avg")
precision_plot([one_feat_acc_pr_df], height=800, more_colours=True)

one_feat_acc_pr_df = one_feat_acc_pr_df.sort_values(by="accuracy_avg")
accuracy_plot([one_feat_acc_pr_df], height=800, more_colours=True)

## Regularisation Experiments

In [ ]:
lr_df, lr_acc_pr_df = train(run_name=f"regularisation_experiments", df=df, model_types=["logistic_regression", "logistic_regression_l1", "logistic_regression_l2","logistic_regression_elastic"])

precision_plot([baseline_df[baseline_df["model"] == "logistic_regression"], lr_acc_pr_df])
accuracy_plot([baseline_df[baseline_df["model"] == "logistic_regression"], lr_acc_pr_df])

## Single Feature Group Models

Compare predictive power of individual feature groups.

In [ ]:
group1_df, group1_acc_pr_df = train(run_name="feature_group1", df=df[[c for c in df.columns if c[-1] == str(1) or c == "Diagnosis"]])
group2_df, group2_acc_pr_df = train(run_name="feature_group2", df=df[[c for c in df.columns if c[-1] == str(2) or c == "Diagnosis"]])
group3_df, group3_acc_pr_df = train(run_name="feature_group3", df=df[[c for c in df.columns if c[-1] == str(3) or c == "Diagnosis"]])

precision_plot([default_acc_pr_df, group1_acc_pr_df, group2_acc_pr_df, group3_acc_pr_df])
accuracy_plot([default_acc_pr_df, group1_acc_pr_df, group2_acc_pr_df, group3_acc_pr_df])

In [ ]:
plot_roc_curves(group1_df)
plot_roc_curves(group2_df)
plot_roc_curves(group3_df)

# Model without top 5 features

In case top 5 features are leading to overfitting.

In [ ]:
minus_top5_df, minus_top5_acc_pr_df = train(run_name="without_top5", df=df.drop(columns=["area3", "perimeter3", "concave_points3", "radius3", "concave_points1"]))

precision_plot([baseline_df, default_acc_pr_df, minus_top5_acc_pr_df])
accuracy_plot([baseline_df, default_acc_pr_df, minus_top5_acc_pr_df])

In [ ]:
plot_roc_curves(minus_top5_df)

## Without top 10 features

In [ ]:
minus_top10_df, minus_top10_acc_pr_df = train(run_name="without_top10", df=df.drop(columns=[
    "area3", "perimeter3", "concave_points3", "radius3", "concave_points1", "concavity3", "area1", "area2", "concavity1", "texture3"
]))

precision_plot([baseline_df, default_acc_pr_df, minus_top10_acc_pr_df])
accuracy_plot([baseline_df, default_acc_pr_df, minus_top10_acc_pr_df])

In [ ]:
plot_roc_curves(minus_top10_df)

## Optimise Models

In [ ]:
r = optimise.run_optuna(df, "random_forest", n_trials=100, n_splits=5, random_state=42)

In [ ]:
run_name = "opt"
rr = pd.DataFrame([{k: v for k, v in r.items() if k != "params"}])
acc_pr_df = rr[['model', 'accuracy_min', 'accuracy_mean', 'accuracy_max', 'precision_min', 'precision_mean', 'precision_max']].copy()
acc_pr_df.rename(columns={"accuracy_mean": "accuracy_avg", "precision_mean": "precision_avg"}, inplace=True)
acc_pr_df[["accuracy_min","accuracy_avg","accuracy_max","precision_min","precision_avg","precision_max"]] *= 100
acc_pr_df['model_type'] = run_name

acc_pr_df.replace(
    {
        "random_forest" : "random_forest_classification",
        "xgboost": "xgboost_classification",
        "svm": "support_vector_classification",
        "neural_network": "neural_network_classification"
    }, inplace=True
)

accuracy_plot(
    [
        baseline_df[baseline_df['model'] == 'random_forest_classification'],
        default_acc_pr_df[default_acc_pr_df['model'] == 'random_forest_classification'],
        acc_pr_df[acc_pr_df['model'] == 'random_forest_classification']
    ]
)
precision_plot(
    [
        baseline_df[baseline_df['model'] == 'random_forest_classification'],
        default_acc_pr_df[default_acc_pr_df['model'] == 'random_forest_classification'],
        acc_pr_df[acc_pr_df['model'] == 'random_forest_classification']
    ]
)

In [ ]:
r = optimise.run_optuna(df, "logistic_regression", n_trials=200, n_splits=5, random_state=42)

In [ ]:
run_name = "opt"
rr = pd.DataFrame([{k: v for k, v in r.items() if k != "params"}])
acc_pr_df = rr[['model', 'accuracy_min', 'accuracy_mean', 'accuracy_max', 'precision_min', 'precision_mean', 'precision_max']].copy()
acc_pr_df.rename(columns={"accuracy_mean": "accuracy_avg", "precision_mean": "precision_avg"}, inplace=True)
acc_pr_df[["accuracy_min","accuracy_avg","accuracy_max","precision_min","precision_avg","precision_max"]] *= 100
acc_pr_df['model_type'] = run_name

acc_pr_df.replace(
    {
        "random_forest" : "random_forest_classification",
        "xgboost": "xgboost_classification",
        "svm": "support_vector_classification",
        "neural_network": "neural_network_classification"
    }, inplace=True
)

accuracy_plot(
    [
        baseline_df[baseline_df['model'] == 'logistic_regression'],
        default_acc_pr_df[default_acc_pr_df['model'] == 'logistic_regression'],
        acc_pr_df[acc_pr_df['model'] == 'logistic_regression']
    ]
)
precision_plot(
    [
        baseline_df[baseline_df['model'] == 'logistic_regression'],
        default_acc_pr_df[default_acc_pr_df['model'] == 'logistic_regression'],
        acc_pr_df[acc_pr_df['model'] == 'logistic_regression']
    ]
)